In [31]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [68]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
from secrets import IEX_CLOUD_API_TOKEN

Testing an API CALL

In [69]:
symbol = "AAPL"
IEX_CLOUD_API_TOKEN = 'sk_2ee3de7590d14840ac2ea5f86d3c26d5'
api_url = f"https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)


{'avgTotalVolume': 60369736, 'calculationPrice': 'close', 'change': -0.35, 'changePercent': -0.00211, 'close': 165.21, 'closeSource': 'official', 'closeTime': 1681502400142, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 165.25, 'delayedPriceTime': 1681502390086, 'extendedChange': -0.06, 'extendedChangePercent': -0.00036, 'extendedPrice': 165.15, 'extendedPriceTime': 1681515961526, 'high': 166.32, 'highSource': '15 minute delayed price', 'highTime': 1681502399994, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 165.2, 'iexCloseTime': 1681502399751, 'iexLastUpdated': 1681502675534, 'iexMarketPercent': 0.012120543922142254, 'iexOpen': 164.56, 'iexOpenTime': 1681479000972, 'iexRealtimePrice': 165.18, 'iexRealtimeSize': 2, 'iexVolume': 598591, 'lastTradeTime': 1681502399752, 'latestPrice': 165.21, 'latestSource': 'Close', 'latestTime': 'April 14, 2023', 'latestUpdate': 1681502400142, 'latestVolume': 49386480, 'low': 163.82, 'lowSource': '15

In [70]:
price = data['latestPrice']
pe_ratio = data['peRatio']
price
pe_ratio

28.1

In [71]:
def chuncks(lst, n):
  for i in range(0, len(lst), n):
    yield lst[i:i+n]


symbol_groups = list(chuncks(stocks['Ticker'], 100))
symbol_strings= []
for i in range(0, len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'Price-to-Earning Ratio', 'Number of Shares To Buy']


In [72]:
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Price-to-Earning Ratio,Number of Shares To Buy


In [ ]:
for symbol_string in symbol_strings:
  batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    final_dataframe = final_dataframe.append(
      pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A'

      ],
      index = my_columns

      ),
        ignore_index=True
    )

final_dataframe

In [74]:
final_dataframe

,Ticker,Price,Price-to-Earning Ratio,Number of Shares To Buy
0,A,139.20,31.42,N/A
1,AAL,12.79,65.96,N/A
2,AAP,123.60,14.95,N/A
3,AAPL,165.21,28.1,N/A
4,ABBV,161.59,24.37,N/A
...,...,...,...,...
496,YUM,134.88,29.51,N/A
497,ZBH,132.37,120.34,N/A
498,ZBRA,303.08,34.4,N/A
499,ZION,29.95,3.09,N/A


Removendo Glamour Stocks

In [ ]:
final_dataframe.sort_values('Price-to-Earning Ratio', ascending=True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earning Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis=1, inplace=True)
final_dataframe


In [87]:
final_dataframe.drop('level_0', axis=1, inplace=True)

In [ ]:
final_dataframe

In [92]:
def portfolio_input():
  global portfolio_size
  portfolio_size = input('Digite o vlaor do Investimento: ')

  try:
    val = float(portfolio_size)

  except ValueError:
    print('Digite em formato de numeral!')
    portfolio_size = float(input("Digite o valor de Investimento: "))

  
    

In [93]:
portfolio_input()

Digite o vlaor do Investimento: 100000


In [107]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
  final_dataframe.loc[row, "Number of Shares To Buy"] = math.floor(position_size/final_dataframe.loc[row, 'Price'])



In [ ]:
final_dataframe

Criando um algoritimo melhor

In [184]:
symbol = "AAPL"
batch_api_url_call = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_url_call).json()

pe_ratio = data[symbol]['quote']['peRatio']

pb_ratio = data['AAPL']['advanced-stats']['priceToBook']
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']


enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda =  data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda


gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit =  enterprise_value / gross_profit

print(ev_to_gross_profit)


16.055733948738848


In [ ]:
#criando o dataframe

rv_columns= [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earning Ratio',
    'PE Percentile',
    'Price-to-book Ratio',
    'PB Percentile',
    'Price-to-sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
  batch_api = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api).json()
  for symbol in symbol_string.split(','):
    enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
    ebitda =  data[symbol]['advanced-stats']['EBITDA']
    gross_profit = data[symbol]['advanced-stats']['grossProfit']

    try:
      ev_to_ebitda = enterprise_value/ebitda 
    except TypeError:
      ev_to_ebitda = np.NaN

# fazendo esses trys e excepts pois algumas empresas não divulgam tais informações. 
# o problema com os valores nulos sera resolvido mais a frente

    try:
      ev_to_gross_profit = enterprise_value/gross_profit
    except TypeError:
      ev_to_gross_profit = np.NaN


    rv_dataframe = rv_dataframe.append(
        pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'NaN',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
            

        ],
        index= rv_columns),
          ignore_index = True
    )


In [140]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.20,NaN,31.42,N/A,7.45,N/A,6.03,N/A,22.080080,N/A,11.507343,N/A,N/A
1,AAL,12.79,NaN,65.96,N/A,-1.46,N/A,0.1726,N/A,9.306850,N/A,0.717812,N/A,N/A
2,AAP,123.60,NaN,14.95,N/A,2.73,N/A,0.6549,N/A,8.426761,N/A,1.694745,N/A,N/A
3,AAPL,165.21,NaN,28.1,N/A,46.18,N/A,6.76,N/A,21.384621,N/A,16.055734,N/A,N/A
4,ABBV,161.59,NaN,24.37,N/A,16.63,N/A,4.94,N/A,12.165593,N/A,8.391984,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,134.88,NaN,29.51,N/A,-4.25,N/A,5.51,N/A,21.272514,N/A,9.653389,N/A,N/A
497,ZBH,132.37,NaN,120.34,N/A,2.3,N/A,3.99,N/A,18.836574,N/A,6.709793,N/A,N/A
498,ZBRA,303.08,NaN,34.4,N/A,5.76,N/A,2.72,N/A,16.469157,N/A,6.728253,N/A,N/A
499,ZION,29.95,NaN,3.09,N/A,1.04,N/A,1.17,N/A,2.500426,N/A,1.205143,N/A,N/A


In [186]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)] # Lidando com valores NaN

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
6,ABMD,381.020,NaN,65.69,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
28,ALXN,182.500,NaN,59.45,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,324.480,NaN,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,319.740,NaN,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,94.920,NaN,47.94,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,NaN,9.73,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
121,CTXS,103.900,NaN,39.96,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.420,NaN,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,49.260,NaN,14.27,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,57.340,NaN,32.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [187]:
for column in ['Price-to-Earning Ratio',
                'Price-to-book Ratio',
            'Price-to-sales Ratio', 'EV/EBITDA',
            'EV/GP']:
  rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

In [188]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)] # não existe mais valores Nan

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


Calculando as Porcentagens


In [189]:
from scipy.stats import percentileofscore as score

metricas = { ## dicionario 
    'Price-to-Earning Ratio':'PE Percentile',
    'Price-to-book Ratio':'PB Percentile',
    'Price-to-sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile',

}
for metric in metricas.keys(): ## percore pelas chaves do dicionario(price-to-earning  etc .. ) 
  for row in rv_dataframe.index: ## percore todas as linhas do data frame 
    rv_dataframe.loc[row, metricas[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
## para cada linha, entra na coluna dos percentiles e calcula o score
## score = porcentagem em comparação aos outros, ex: se o numero for maior que 80%
## dos outros valores da coluna, então recebe 80%



In [151]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.20,NaN,31.42,72.954092,7.45,76.846307,6.0300,84.830339,22.080080,85.429142,11.507343,84.231537,N/A
1,AAL,12.79,NaN,65.96,95.409182,-1.46,6.187625,0.1726,0.998004,9.306850,26.347305,0.717812,2.59481,N/A
2,AAP,123.60,NaN,14.95,32.135729,2.73,44.91018,0.6549,9.580838,8.426761,22.355289,1.694745,11.576846,N/A
3,AAPL,165.21,NaN,28.10,67.664671,46.18,97.804391,6.7600,86.626747,21.384621,83.632735,16.055734,93.413174,N/A
4,ABBV,161.59,NaN,24.37,60.479042,16.63,93.213573,4.9400,78.043912,12.165593,40.718563,8.391984,67.664671,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,134.88,NaN,29.51,70.858283,-4.25,5.788423,5.5100,82.235529,21.272514,83.233533,9.653389,75.0499,N/A
497,ZBH,132.37,NaN,120.34,98.003992,2.30,37.62475,3.9900,71.257485,18.836574,75.848303,6.709793,48.902196,N/A
498,ZBRA,303.08,NaN,34.40,78.043912,5.76,69.760479,2.7200,53.692615,16.469157,67.265469,6.728253,49.101796,N/A
499,ZION,29.95,NaN,3.09,8.982036,1.04,12.075848,1.1700,20.858283,2.500426,2.994012,1.205143,7.185629,N/A


Calculando o rv score

In [190]:
from statistics import mean

for row in rv_dataframe.index: ## percorre todas as linhas do tada frame
  value_percentiles = [] ## lista para pegar todas as procentagens da linha
  for metric in metricas.keys():
    value_percentiles.append(rv_dataframe.loc[row, metricas[metric]]) ## adicionando todas as porcentagens na lista
  
  
  rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles) ## rv score da linha = media das porcentagens

In [157]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.20,NaN,31.42,72.954092,7.45,76.846307,6.0300,84.830339,22.080080,85.429142,11.507343,84.231537,80.858283
1,AAL,12.79,NaN,65.96,95.409182,-1.46,6.187625,0.1726,0.998004,9.306850,26.347305,0.717812,2.59481,26.307385
2,AAP,123.60,NaN,14.95,32.135729,2.73,44.91018,0.6549,9.580838,8.426761,22.355289,1.694745,11.576846,24.111776
3,AAPL,165.21,NaN,28.10,67.664671,46.18,97.804391,6.7600,86.626747,21.384621,83.632735,16.055734,93.413174,85.828343
4,ABBV,161.59,NaN,24.37,60.479042,16.63,93.213573,4.9400,78.043912,12.165593,40.718563,8.391984,67.664671,68.023952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,134.88,NaN,29.51,70.858283,-4.25,5.788423,5.5100,82.235529,21.272514,83.233533,9.653389,75.0499,63.433134
497,ZBH,132.37,NaN,120.34,98.003992,2.30,37.62475,3.9900,71.257485,18.836574,75.848303,6.709793,48.902196,66.327345
498,ZBRA,303.08,NaN,34.40,78.043912,5.76,69.760479,2.7200,53.692615,16.469157,67.265469,6.728253,49.101796,63.572854
499,ZION,29.95,NaN,3.09,8.982036,1.04,12.075848,1.1700,20.858283,2.500426,2.994012,1.205143,7.185629,10.419162


Selecionando as 50 melhores ações

In [191]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
283,LNC,20.97,NaN,-1.60,7.984032,0.9791,10.978044,0.1904,1.397206,-1.761020,1.996008,0.190432,0.199601,4.510978
191,FRC,13.12,NaN,1.59,8.582834,0.1770,6.586826,0.3621,4.99002,1.048004,2.195609,0.367889,0.399202,4.550898
270,KSS,23.38,NaN,-147.69,1.197605,0.6742,7.984032,0.1402,0.598802,6.889149,15.568862,1.093384,5.788423,6.227545
202,GM,34.49,NaN,5.63,12.37525,0.7104,8.582834,0.3072,3.592814,1.546249,2.39521,1.162679,6.387226,6.666667
18,AIG,51.44,NaN,3.96,9.381238,0.9584,10.578842,0.6710,10.379242,1.958826,2.59481,0.671048,2.39521,7.065868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,IDXX,482.49,NaN,60.09,93.612774,65.4300,98.403194,11.8300,96.207585,40.630287,97.005988,20.487368,98.003992,96.646707
85,CDNS,214.55,NaN,69.43,95.608782,21.5800,95.00998,16.6300,99.001996,54.107185,98.403194,18.524985,96.606786,96.926148
337,NOW,463.03,NaN,289.39,99.600798,19.5100,94.011976,13.5500,97.804391,121.010787,99.600798,16.811795,94.610778,97.125749
149,DXCM,115.70,NaN,144.97,98.203593,21.0800,94.810379,15.4400,98.403194,111.571531,99.401198,23.634371,99.201597,98.003992


In [195]:
rv_dataframe = rv_dataframe[(rv_dataframe['Price-to-Earning Ratio']>0) & (rv_dataframe['Price-to-book Ratio']>0) & (rv_dataframe['EV/EBITDA'] > 0) & (rv_dataframe['EV/GP'] > 0)]

In [196]:
rv_dataframe = rv_dataframe[:50]

In [197]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
191,FRC,13.12,NaN,1.59,8.582834,0.1770,6.586826,0.3621,4.99002,1.048004,2.195609,0.367889,0.399202,4.550898
202,GM,34.49,NaN,5.63,12.37525,0.7104,8.582834,0.3072,3.592814,1.546249,2.39521,1.162679,6.387226,6.666667
18,AIG,51.44,NaN,3.96,9.381238,0.9584,10.578842,0.6710,10.379242,1.958826,2.59481,0.671048,2.39521,7.065868
457,UNM,40.03,NaN,6.16,13.373253,0.8565,9.381238,0.6570,9.780439,3.544797,4.99002,0.656989,2.195609,7.944112
424,SYF,29.80,NaN,4.85,10.978044,1.0500,12.57485,0.6791,10.578842,2.911408,3.592814,0.827809,3.592814,8.263473
75,C,49.56,NaN,7.11,14.770459,0.5046,7.185629,0.9051,15.169661,3.985331,6.187625,0.953393,4.191617,9.500998
106,COF,96.58,NaN,5.39,11.976048,0.6952,8.383234,0.9526,16.167665,3.929849,5.988024,1.124851,5.988024,9.700599
499,ZION,29.95,NaN,3.09,8.982036,1.0400,12.075848,1.1700,20.858283,2.500426,2.994012,1.205143,7.185629,10.419162
136,DISH,7.84,NaN,2.17,8.782435,0.1361,6.387226,0.1464,0.798403,7.655204,19.560878,3.987949,27.944112,12.694611
364,PFG,75.89,NaN,4.03,9.580838,1.8200,27.844311,1.0400,17.764471,2.869806,3.393214,1.042814,4.99002,12.714571


In [198]:
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,13.12,NaN,1.59,8.582834,0.1770,6.586826,0.3621,4.99002,1.048004,2.195609,0.367889,0.399202,4.550898
1,GM,34.49,NaN,5.63,12.37525,0.7104,8.582834,0.3072,3.592814,1.546249,2.39521,1.162679,6.387226,6.666667
2,AIG,51.44,NaN,3.96,9.381238,0.9584,10.578842,0.6710,10.379242,1.958826,2.59481,0.671048,2.39521,7.065868
3,UNM,40.03,NaN,6.16,13.373253,0.8565,9.381238,0.6570,9.780439,3.544797,4.99002,0.656989,2.195609,7.944112
4,SYF,29.80,NaN,4.85,10.978044,1.0500,12.57485,0.6791,10.578842,2.911408,3.592814,0.827809,3.592814,8.263473
5,C,49.56,NaN,7.11,14.770459,0.5046,7.185629,0.9051,15.169661,3.985331,6.187625,0.953393,4.191617,9.500998
6,COF,96.58,NaN,5.39,11.976048,0.6952,8.383234,0.9526,16.167665,3.929849,5.988024,1.124851,5.988024,9.700599
7,ZION,29.95,NaN,3.09,8.982036,1.0400,12.075848,1.1700,20.858283,2.500426,2.994012,1.205143,7.185629,10.419162
8,DISH,7.84,NaN,2.17,8.782435,0.1361,6.387226,0.1464,0.798403,7.655204,19.560878,3.987949,27.944112,12.694611
9,PFG,75.89,NaN,4.03,9.580838,1.8200,27.844311,1.0400,17.764471,2.869806,3.393214,1.042814,4.99002,12.714571


Numero de ações para comprar 

In [199]:
portfolio_input()

Digite o vlaor do Investimento: 100000000


In [200]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
position_size

2000000.0

In [201]:
for row in rv_dataframe.index:
  rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])

In [203]:

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,13.12,152439,1.59,8.582834,0.1770,6.586826,0.3621,4.99002,1.048004,2.195609,0.367889,0.399202,4.550898
1,GM,34.49,57987,5.63,12.37525,0.7104,8.582834,0.3072,3.592814,1.546249,2.39521,1.162679,6.387226,6.666667
2,AIG,51.44,38880,3.96,9.381238,0.9584,10.578842,0.6710,10.379242,1.958826,2.59481,0.671048,2.39521,7.065868
3,UNM,40.03,49962,6.16,13.373253,0.8565,9.381238,0.6570,9.780439,3.544797,4.99002,0.656989,2.195609,7.944112
4,SYF,29.80,67114,4.85,10.978044,1.0500,12.57485,0.6791,10.578842,2.911408,3.592814,0.827809,3.592814,8.263473
5,C,49.56,40355,7.11,14.770459,0.5046,7.185629,0.9051,15.169661,3.985331,6.187625,0.953393,4.191617,9.500998
6,COF,96.58,20708,5.39,11.976048,0.6952,8.383234,0.9526,16.167665,3.929849,5.988024,1.124851,5.988024,9.700599
7,ZION,29.95,66777,3.09,8.982036,1.0400,12.075848,1.1700,20.858283,2.500426,2.994012,1.205143,7.185629,10.419162
8,DISH,7.84,255102,2.17,8.782435,0.1361,6.387226,0.1464,0.798403,7.655204,19.560878,3.987949,27.944112,12.694611
9,PFG,75.89,26353,4.03,9.580838,1.8200,27.844311,1.0400,17.764471,2.869806,3.393214,1.042814,4.99002,12.714571


Transformando esses dados em um excel para futuras pesquisas


In [204]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name ='Compras', index=False)
writer.save()

<ipython-input-204-ec70628d3535>:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
